In [1]:
#:pip install datasets 

## Training

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Load the dataset:

In [3]:
import pandas as pd


train_df = pd.read_csv('../data/markup2train.csv')
test_df = pd.read_csv('../data/markup2test.csv')
train_df.head(3)

,Unnamed: 0,index,path,original,new,var
0,118,131,0024c525-9f48-4e21-8eec-d00aea56deaa,В целях продвижения нового канала продаж «Цифр...,Цель продвижения канала продаж «Цифровой офис...,4
1,1119,1184,00e3fe8f-114e-4b83-a15f-aa7af38f931a,в разрезе центров прибыли верхнего уровня:,по прибыли верхнего уровня:,1
2,1033,1123,00e19c69-1d75-4459-b4e3-8256a6fbdaf3,в срок до 16.12.2021 включительно сформировать...,до 16.12.2021 определить победителей Акции;,4


In [4]:
train_df=train_df[train_df['new'].isna()==False].reset_index()
train_df.shape

(5841, 7)

In [5]:
from datasets import Dataset
from sklearn.model_selection import train_test_split


train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(val_df)

2022-05-27 12:37:44.664787: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


Load the backbone:

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer

backbone_id = "sberbank-ai/rugpt3large_based_on_gpt2"

model = AutoModelForCausalLM.from_pretrained(backbone_id)
tokenizer = AutoTokenizer.from_pretrained(backbone_id, pad_token="<pad>", eos_token="<pad>")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Define the prompt format:

In [7]:
from ruprompts import PromptFormat

prompt_format = PromptFormat("<P*100>{original}<P*20>")

Define the parametrization of trainable embeddings:

In [8]:
from ruprompts import TensorPromptProvider
from transformers import set_seed

set_seed(1)

prompt_provider = TensorPromptProvider()

Compose prompt format and prompt provider into prompt object and apply it to the model and tokenizer, i.e. add special tokens to the tokenizer and modify the layer of input embeddings of the model:

In [9]:
from ruprompts import Prompt

prompt = Prompt(prompt_format, prompt_provider)
prompt.patch(model, tokenizer)

Preprocess the data:
1. format the data entries with the specified prompt format
2. tokenize the resulting sequences
3. truncate the `truncation_field` if sequence length exceeds `max_tokens`

In [10]:
from ruprompts import Text2TextPreprocessor

preprocessor = Text2TextPreprocessor(
    prompt_format=prompt_format,
    tokenizer=tokenizer,
    target_field="new",
    truncation_field="original",
)

train_dataset = train_dataset.map(preprocessor)
valid_dataset = valid_dataset.map(preprocessor)

  0%|          | 0/5256 [00:00<?, ?ex/s]

  0%|          | 0/585 [00:00<?, ?ex/s]

Define training arguments:

In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="promts_checkpoints",
    overwrite_output_dir=True,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    metric_for_best_model="eval_loss",
    learning_rate=0.1,
    num_train_epochs=15,
    seed=0
)

Choose optimization options:

In [12]:
from transformers.optimization import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(prompt_provider.parameters(), lr=training_args.learning_rate)

/home/user/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Define the callbacks and start training:

In [13]:
from transformers import Trainer
from ruprompts.callbacks import (
    FreezeTransformerUnfreezePrompt,
    ReduceCheckpoint,
    SavePretrainedPrompt,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=preprocessor.collate_fn(),
    optimizers=(optimizer, None),
    callbacks=[FreezeTransformerUnfreezePrompt(), ReduceCheckpoint(), SavePretrainedPrompt(prompt)],
)

trainer.train()

/home/user/conda/lib/python3.7/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, '__version__') or LooseVersion(tensorboard.__version__) < LooseVersion('1.15'):
The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: new, Unnamed: 0, path, var, level_0, original, index, __index_level_0__. If new, Unnamed: 0, path, var, level_0, original, index, __index_level_0__ are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 5256
  Num Epochs = 15
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 9855


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch,Training Loss,Validation Loss
1,1.158400,0.578571
2,0.649600,0.539967
3,0.603800,0.505845
4,0.577900,0.486589
5,0.559900,0.477013
6,0.547500,0.462273
7,0.538400,0.460558
8,0.531100,0.455532
9,0.522400,0.453796
10,0.514700,0.447421


The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: new, Unnamed: 0, path, var, level_0, original, index, __index_level_0__. If new, Unnamed: 0, path, var, level_0, original, index, __index_level_0__ are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 585
  Batch size = 8
Saving model checkpoint to promts_checkpoints/checkpoint-657
Configuration saved in promts_checkpoints/checkpoint-657/config.json
Model weights saved in promts_checkpoints/checkpoint-657/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: new, Unnamed: 0, path, var, level_0, original, index, __index_level_0__. If new, Unnamed: 0, path, var, level_0, original, index, __index_level_0__ are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message

TrainOutput(global_step=9855, training_loss=0.5794558791688467, metrics={'train_runtime': 1848.8938, 'train_samples_per_second': 42.642, 'train_steps_per_second': 5.33, 'total_flos': 1.339784227196928e+16, 'train_loss': 0.5794558791688467, 'epoch': 15.0})

## Inference

Load prompt from the last checkpoint:

In [17]:
training_args.max_steps

-1

In [14]:
from transformers import pipeline

prompt = Prompt.from_pretrained(f"promts_checkpoints/checkpoint-9855")

ppln = pipeline("text2text-generation-with-prompt", prompt=prompt, model=model, tokenizer=tokenizer, device=0)

In [15]:
test_text = ['Начиная с премирования по итогам работы за январь 2021 г., руководствоваться перечнем операций и коэффициентами пересчета продуктов в условные продукты (далее - УП) для менеджеров по продажам в соответствии с Приложением к настоящему Распоряжению.',
            'В случае подтверждения устранения Катастрофической или Серьезной ошибки Заказчик самостоятельно осуществляет тиражирование ПО в своих подразделениях и филиалах.',
            'Контроль за исполнением настоящего Распоряжения оставляю за собой.',
            'Утвердить перечень автоматизированных систем и информационных ресурсов Банка, доступных к подключению Cотрудникам (Приложение 1).']

Run inference:

In [16]:
from tqdm import tqdm
import transformers

transformers.logging.set_verbosity_error()
beam_count = 10

for test_string in test_text:
    
    options = ppln(
        {"original": test_string},
        do_sample=False,
        num_return_sequences=1,   # unk
        num_beams=beam_count,)
    print(f"ORIGINAL: {test_string}")
    print("REPHRASED: ",options[0]['generated_text'])
    print("\n####", end='\n\n')

2022-05-30 12:21:15.991761: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


ORIGINAL: Начиная с премирования по итогам работы за январь 2021 г., руководствоваться перечнем операций и коэффициентами пересчета продуктов в условные продукты (далее - УП) для менеджеров по продажам в соответствии с Приложением к настоящему Распоряжению.
REPHRASED:  Начиная с премирования по итогам работы за январь 2021 г., руководствоваться перечнем операций и коэффициентами пересчета продуктов в условные продукты (далее - УП) для менеджеров по продажам по Приложению к этому Распоряжению.

####

ORIGINAL: В случае подтверждения устранения Катастрофической или Серьезной ошибки Заказчик самостоятельно осуществляет тиражирование ПО в своих подразделениях и филиалах.
REPHRASED:  При подтверждении устранения Катастрофической или Серьезной ошибки Заказчик самостоятельно тиражирует ПО в подразделениях и филиалах.

####

ORIGINAL: Контроль за исполнением настоящего Распоряжения оставляю за собой.
REPHRASED:  Исполнение этого Распоряжения оставляю за собой.

####

ORIGINAL: Утвердить перече